In [1]:
import os
import sys
os.environ["SIMCORE_CONFIG_PATH"] = r"C:\Users\anderegg\Documents\dev\OSPARC\osparc-simcore-ssh\services\dy-2Dgraph\use-cases\cc-twod\cc_config-twod.json"
sys.path.append(r'C:\Users\anderegg\Documents\dev\OSPARC\osparc-simcore-ssh\packages\simcore-sdk\src')

In [2]:
from simcore_sdk.nodeports import PORTS

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import tqdm

number_of_columns = 165

In [4]:
data_path = PORTS.inputs[0].get()

# get the list of files
dat_files = sorted([os.path.join(data_path, x) for x in os.listdir(data_path) if x.endswith(".dat")], key=lambda f: int(''.join(filter(str.isdigit, f))))
out_images_path = os.path.join(data_path, r"tmp")


In [5]:
def plot_contour(i):
    plt.clf()
    data_frame = pd.read_csv(dat_files[i], sep='\t', header=None, usecols=range(0,number_of_columns))
    plt.contourf(data_frame.values, cmap=plt.get_cmap('jet'), levels=np.arange(-100.0, 51.0, 1.0))
    plt.axis("off")
    plt.colorbar()



# create movie writer
FFMpegWriter = animation.writers["ffmpeg"]
metdata = dict(title="Action potentials", artist="", comment="")
movie_writer = FFMpegWriter(fps=30, metadata=metdata)

pixel_size = 300
dpi = 96.0
plt.ioff()
fig = plt.figure(frameon=False, figsize=(pixel_size/dpi, pixel_size/dpi), dpi=dpi)


number_of_frames = len(dat_files)
video_file_path = os.path.join(out_images_path, "test_movie.mp4")
with movie_writer.saving(fig, video_file_path, dpi):
    for frame in tqdm.tqdm(range(0, number_of_frames)):
        plot_contour(frame)
        movie_writer.grab_frame()
plt.close(fig)

        
import io
import base64
from IPython.display import HTML
video = io.open(video_file_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:07<00:00,  7.36it/s]
